<a href="https://colab.research.google.com/github/BakrAsskali/AE-VAE-GANs/blob/main/AE_and_VAE_continuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch;
torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from sklearn.model_selection import ParameterGrid
import itertools

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_dims):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, latent_dims)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        return self.linear2(x)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 784)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.reshape((-1, 1, 28, 28))

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:
def train(autoencoder, data, epochs=20):
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):
        for x, y in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum()
            loss.backward()
            opt.step()
    return autoencoder

In [ ]:
# use grid search to find the best hyperparameters for the model
grid_search = {
    "learning_rate": [0.001, 0.01, 0.1],
    "batch_size": [32, 64, 128],
    "epochs": [10, 20, 30]
}
# create a list of hyperparameters
hyperparameters = list(ParameterGrid(grid_search))
# create a list of all possible combinations of hyperparameters
combinations = list(itertools.product(hyperparameters, repeat = 1))

In [ ]:
# create a list to store the loss values
loss_values = []
data = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST('./data',
               transform=torchvision.transforms.ToTensor(),
               download=True),
        batch_size=128,
        shuffle=True)
# loop through all the combinations of hyperparameters
for combination in combinations:
    # create a model
    model = Autoencoder(2).to(device)
    # create a loss function
    criterion = torch.nn.MSELoss()
    # create an optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr = combination[0]["learning_rate"])
    # loop through the number of epochs
    for epoch in range(combination[0]["epochs"]):
        # create a list to store the losses
        losses = []
        # loop through the data loader
        for batch_features, _ in data:
            # reshape the data
            batch_features = batch_features.view(-1, 28 * 28)
            # set the gradients to zero
            optimizer.zero_grad()
            # forward pass
            outputs = model(batch_features)
            # Reshape the outputs tensor to match the shape of the batch_features tensor
            outputs = outputs.view(-1, 784)

            # Calculate the loss
            loss = criterion(outputs, batch_features)
            # backward pass
            loss.backward()
            # update the weights
            optimizer.step()
            # append the loss
            losses.append(loss.item())
        # print the loss
        print(f"Epoch {epoch + 1}, loss: {np.mean(losses)}")
    # append the loss values
    loss_values.append(np.mean(losses))

Epoch 1, loss: 0.05591150215153755
Epoch 2, loss: 0.04717671284988237
Epoch 3, loss: 0.04539329800873931
Epoch 4, loss: 0.04446853827565972
Epoch 5, loss: 0.04380475542247931
Epoch 6, loss: 0.0432480101678163
Epoch 7, loss: 0.042810471120800796
Epoch 8, loss: 0.042416814881474224
Epoch 9, loss: 0.04205711309843734
Epoch 10, loss: 0.041730828726215405
Epoch 1, loss: 0.05012139605719652
Epoch 2, loss: 0.045064932128577345
Epoch 3, loss: 0.0439535023164012
Epoch 4, loss: 0.04325635404760904
Epoch 5, loss: 0.04279063554651447
Epoch 6, loss: 0.042420870340518606
Epoch 7, loss: 0.04210429635446972
Epoch 8, loss: 0.041886765636932624
Epoch 9, loss: 0.04172585887123527
Epoch 10, loss: 0.041592236227000444
Epoch 1, loss: 0.11226055998283663
Epoch 2, loss: 0.11199256120078853
Epoch 3, loss: 0.1119937520227961
Epoch 4, loss: 0.11199099287740204
Epoch 5, loss: 0.1119877586740929
Epoch 6, loss: 0.1119933707722977
Epoch 7, loss: 0.11199420739783407
Epoch 8, loss: 0.11199301722715659
Epoch 9, loss: 0

In [ ]:
# find the best hyperparameters
best_hyperparameters = hyperparameters[np.argmin(loss_values)]
# print the best hyperparameters
print(best_hyperparameters)

In [ ]:
latent_dims = 2
autoencoder = Autoencoder(latent_dims).to(device) # GPU

data = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST('./data',
               transform=torchvision.transforms.ToTensor(),
               download=True),
        batch_size=128,
        shuffle=True)

autoencoder = train(autoencoder, data, best_hyperparameters["epochs"])